### Mike Ogrysko
### CS 737 Machine Learning
Multilayer Perception implementation
- 2 hidden layer neural network implementation
- Uses MNIST dataset

In [1]:
import numpy as np
from numpy.random import ranf
from sklearn.metrics import confusion_matrix

In [4]:
#function to load MNIST dataset
def load_mnist(path, kind='train'):
    from numpy import fromfile, uint8
    import os
    import struct

    labels_path = os.path.join(path, '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte' % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = fromfile(lbpath, dtype=uint8)
        with open(images_path, 'rb') as imgpath:
            magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
            images = fromfile(imgpath, dtype=uint8).reshape(len(labels), 784)
            images = ((images / 255.) - .5) * 2
    return images, labels


In [6]:
#training and test sets
X_train, y_train = load_mnist('mnist/', kind='train')
print(f'Rows= {X_train.shape[0]}, columns= {X_train.shape[1]}')

X_test, y_test = load_mnist('mnist/', kind='t10k')
print(f'Rows= {X_test.shape[0]}, columns= {X_test.shape[1]}')


Rows= 60000, columns= 784
Rows= 10000, columns= 784


In [7]:
#function to get accuracy
def get_acc(_y_test, _y_pred):
    return ((np.sum(_y_test == _y_pred)).astype(float) / _y_test.shape[0])


In [8]:
#neural network
class NeuralNetMLP(object):

    def __init__(self, n_hidden=30, epochs=300, eta=0.0001, minibatch_size=1, seed=None):
        self.random = np.random.RandomState(seed)  # used to randomize weights
        self.n_hidden = n_hidden  # size of the hidden layer
        self.epochs = epochs  # number of iterations
        self.eta = eta  # learning rate
        self.minibatch_size = minibatch_size  # size of training batch - 1 would not work

    @staticmethod
    def onehot(y, n_classes):  # one hot encode the input class y
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.0
        return onehot.T

    @staticmethod
    def sigmoid(z):  # Eq 1
        return 1.0 / (1.0 + np.exp(-np.clip(z, -250, 250)))

    def _forward(self, X):  # Eq 2
        #input to hidden layer 1
        z_h_1 = np.dot(X, self.w_h_1)
        a_h_1 = self.sigmoid(z_h_1)
        #hidden layer 1 to hidden layer 2
        z_h_2 = np.dot(a_h_1, self.w_h_2)
        a_h_2 = self.sigmoid(z_h_2)
        #hidden layer 2 to output
        z_out = np.dot(a_h_2, self.w_out)
        a_out = self.sigmoid(z_out)
        return z_h_1, a_h_1, z_h_2, a_h_2, z_out, a_out

    @staticmethod
    def compute_cost(y_enc, output):  # Eq 4
        term1 = -y_enc * (np.log(output))
        term2 = (1.0 - y_enc) * np.log(1.0 - output)
        cost = np.sum(term1 - term2)
        return cost

    def predict(self, X):
        z_h_1, a_h_1, z_h_2, a_h_2, z_out, a_out = self._forward(X)
        y_pred = np.argmax(z_out, axis=1)
        return y_pred

    def fit(self, X_train, y_train, X_valid, y_valid):
        import sys
        n_output = np.unique(y_train).shape[0]  # number of class labels
        n_features = X_train.shape[1]
        #weights output
        self.w_out = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden, n_output))
        #weights hidden layer 2
        self.w_h_2 = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden, self.n_hidden))
        #weights hidden layer 1
        self.w_h_1 = self.random.normal(loc=0.0, scale=0.1, size=(n_features, self.n_hidden))
        y_train_enc = self.onehot(y_train, n_output)  # one-hot encode original y
        for i in range(self.epochs):  # Ideally must shuffle at every epoch
            indices = np.arange(X_train.shape[0])
            for start_idx in range(0, indices.shape[0] - self.minibatch_size + 1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]

                z_h_1, a_h_1, z_h_2, a_h_2, z_out, a_out = self._forward(X_train[batch_idx])  # neural network model

                #back propogation of error
                #sigmoid der for a_h_1
                sigmoid_derivative_h_1 = a_h_1 * (1.0 - a_h_1)  # Eq 3
                #sigmoid der for a_h_2
                sigmoid_derivative_h_2 = a_h_2 * (1.0 - a_h_2)  # Eq 3
                delta_out = a_out - y_train_enc[batch_idx]  # Eq 5
                #error at hidden layer
                delta_h_2 = (np.dot(delta_out, self.w_out.T) * sigmoid_derivative_h_2)  # Eq 6
                delta_h_1 = (np.dot(delta_h_2, self.w_h_2.T) * sigmoid_derivative_h_1)  # Eq 6
                #gradient
                grad_w_out = np.dot(a_h_2.T, delta_out)  # Eq 7
                grad_w_h_2 = np.dot(a_h_1.T, delta_h_2)  # Eq 8
                grad_w_h_1 = np.dot(X_train[batch_idx].T, delta_h_1)  # Eq 8
                #update weights
                self.w_out -= self.eta * grad_w_out  # Eq 9
                self.w_h_2 -= self.eta * grad_w_h_2  # Eq 9
                self.w_h_1 -= self.eta * grad_w_h_1  # Eq 9

            # Evaluation after each epoch during training
            z_h_1, a_h_1, z_h_2, a_h_2, z_out, a_out = self._forward(X_train)
            cost = self.compute_cost(y_enc=y_train_enc, output=a_out)
            y_train_pred = self.predict(X_train)  # monitoring training progress through reclassification
            y_valid_pred = self.predict(X_valid)  # monitoring training progress through validation
            train_acc = ((np.sum(y_train == y_train_pred)).astype(float) / X_train.shape[0])
            valid_acc = ((np.sum(y_valid == y_valid_pred)).astype(float) / X_valid.shape[0])
            sys.stderr.write('\r%d/%d | Cost: %.2f ' '| Train/Valid Acc.: %.2f%%/%.2f%% ' %
                             (i + 1, self.epochs, cost, train_acc * 100, valid_acc * 100))
            sys.stderr.flush()
        return self


In [9]:
# Define and fit the neural network
nn = NeuralNetMLP(n_hidden=20, epochs=300, eta=0.0001, minibatch_size=1000, seed=1)

nn.fit(X_train=X_train[:55000], y_train=y_train[:55000], X_valid=X_train[55000:], y_valid=y_train[55000:]) ;

y_pred = nn.predict(X_test)


300/300 | Cost: 14601.96 | Train/Valid Acc.: 96.29%/95.72% 

In [10]:
#get accuracy and confusion matrix
print(f'Accuracy= {get_acc(y_test,y_pred)*100:.2f}%')
print(confusion_matrix(y_test,y_pred))


Accuracy= 94.80%
[[ 960    0    1    1    3    5    7    1    2    0]
 [   0 1114    4    1    0    2    3    4    7    0]
 [  14    2  967    8   10    4    6   10   11    0]
 [   1    1   21  945    1   16    1    9   11    4]
 [   1    0    3    0  941    0    9    2    9   17]
 [  12    2    2   24    0  805   16    1   23    7]
 [   8    3    1    0    8    8  925    0    5    0]
 [   1    8   22    6    8    1    0  971    2    9]
 [   4    4    2   13    6   12    7    4  916    6]
 [   6    6    1   13   25    6    1    8    7  936]]
